In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn
!pip install gdown

### **1. IMPORT LIBRARY**

In [ ]:
#Third-party libraries import
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import dask.dataframe as dd
import gdown

### **2. ACQUIRE DATA**
- **Note**: /Data to your directory

In [ ]:
file_path = r"/content/drive/MyDrive/Data/"

In [ ]:
url = 'https://drive.google.com/uc?id=1Fsy5RcDWAltdkD6rFLZmkqH8AsmQKNQl'
output = 'Data.csv'
gdown.download(url, output, quiet=False)

In [ ]:
reader = pd.read_csv("Data.csv", chunksize=1000000)

In [ ]:
data = pd.concat([chunk for chunk in reader])

### **3. WRANGLE DATA**

#### **A. RFM MODEL FUNCTIONS**

In [ ]:
def create_data_by_year_filter(dataframe, year):
  return dataframe[(dataframe['DATE_DIM'] >= f'{year}-01-01') & (dataframe['DATE_DIM'] <= f'{year}-12-31')]

In [ ]:
def calculate_Q1_median_Q3(dataframe, column):
  Q1 = np.percentile(dataframe[column], 25)
  Q3 = np.percentile(dataframe[column], 75)
  median = np.median(dataframe[column])
  return Q1, median, Q3

In [ ]:
def calculate_upper_limit_lower_limit(dataframe, column, Q1, Q3):
  maximum = float(Q3 + 1.5 * (Q3 - Q1))
  minimum = float(np.min(dataframe[column]))
  return maximum, minimum

In [ ]:
def capping_outlier(value, maximum, minimum):
  if value > maximum:
    return maximum
  elif value < minimum:
    return minimum
  else:
    return value

In [ ]:
def handle_outliers_on_columns(dataframe):
  for column in dataframe.columns:
    Q1, median, Q3 = calculate_Q1_median_Q3(dataframe, column)
    maximum, minimum = calculate_upper_limit_lower_limit(dataframe, column, Q1, Q3)
    dataframe[column] = dataframe[column].apply(capping_outlier, args=(maximum, minimum))
    dataframe = calculate_score(dataframe, column, Q1, median, Q3)
  return dataframe

In [ ]:
def calculate_recency(dataframe, year):
  ref_date = f'{year}-12-31'
  dataframe['DATE_DIM'] = pd.to_datetime(dataframe['DATE_DIM'])
  ref_date = pd.to_datetime(ref_date)
  recency = pd.DataFrame(dataframe.groupby(['BET_ACCOUNT_NUM_HASH']).agg({'DATE_DIM' : lambda x : ((ref_date - x.max()).days)}).reset_index(drop=True))
  recency.rename(columns={'DATE_DIM': 'Recency'}, inplace=True)
  return recency

In [ ]:
def calculate_frequency(dataframe):
  freq = pd.DataFrame(dataframe.groupby('BET_ACCOUNT_NUM_HASH')['DATE_DIM'].count()).reset_index(drop=True)
  freq.rename(columns={'DATE_DIM': 'Frequency'}, inplace=True)
  return freq

In [ ]:
def calculate_monetary(dataframe):
  monetary = pd.DataFrame(dataframe.groupby('BET_ACCOUNT_NUM_HASH')['TOTAL_TURNOVER'].sum()).reset_index(drop=True)
  monetary.rename(columns={'TOTAL_TURNOVER': 'Monetary'}, inplace=True)
  return monetary

In [ ]:
def calculate_RFM(dataframe, year):
  df_year = create_data_by_year_filter(dataframe, year)
  recency = calculate_recency(df_year, year)
  frequency = calculate_frequency(df_year)
  monetary = calculate_monetary(df_year)
  return recency, frequency, monetary, df_year

In [ ]:
def calculate_score(dataframe, column, Q1, median, Q3):
  if column == 'Recency':
    dataframe[f'{column}_Score'] = dataframe[column].apply(lambda x: 4 if x <= Q1 else (3 if x <= median else (2 if x <= Q3 else 1)))
  else:
    dataframe[f'{column}_Score'] = dataframe[column].apply(lambda x: 1 if x <= Q1 else (2 if x <= median else (3 if x <= Q3 else 4)))
  return dataframe

In [ ]:
def segment_score_2021(value):
  if value > 10:
    return 'Champions'
  elif value > 8:
    return 'Loyal Customers'
  elif value > 5:
    return 'About to sleep'
  elif value > 3:
    return 'Hibernating'
  else:
    return 'Lost'

def segment_score_2022(value):
  if value > 10:
    return 'Champions'
  elif value > 8:
    return 'Loyal Customers'
  elif value > 3:
    return 'About to sleep'
  else:
    return 'Hibernating'

In [ ]:
def create_RFM(dataframe, year):
  recency, frequency, monetary, df_year = calculate_RFM(dataframe, year)
  RFM = pd.concat([recency, frequency, monetary], axis = 1)
  RFM = handle_outliers_on_columns(RFM)
  RFM['Overall_Score'] = RFM[['Recency_Score', 'Frequency_Score', 'Monetary_Score']].sum(axis=1)
  Overall_score = pd.DataFrame(RFM.groupby('Overall_Score')[['Recency', 'Frequency', 'Monetary']].mean())
  RFM["BET_ACCOUNT_NUM_HASH"] = list(df_year.groupby('BET_ACCOUNT_NUM_HASH').groups.keys())
  if year == '2021':
    RFM['Segment'] = RFM['Overall_Score'].apply(segment_score_2021)
  elif year == '2022':
    RFM['Segment'] = RFM['Overall_Score'].apply(segment_score_2022)
  RFM.drop(columns = ['Recency', 'Frequency',	'Monetary', 'Recency_Score', 'Frequency_Score' ,'Monetary_Score', 'Overall_Score']	, inplace = True)
  return RFM, Overall_score

#### **B. PREPROCESSING FUNCTIONS**

In [ ]:
def transform_columns_in_batches(data, features, transformer, batch_size=500000):
    num_batches = len(data) // batch_size + 1
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(data))
        data_transform = data.iloc[start_idx:end_idx]
        data_transform[features] = transformer.fit_transform(data_transform)
        data.iloc[start_idx:end_idx] = data_transform

    return data

In [ ]:
def apply_function_in_batches(df, func, batch_size=50000):
    num_batches = len(df) // batch_size + 1
    dfs = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        df_batch = df.iloc[start_idx:end_idx].copy()
        df_batch = func(df_batch)
        dfs.append(df_batch)

    return pd.concat(dfs, ignore_index=True)

In [ ]:
def handle_column(dict, column_to_handle, dataframe):
  dataframe[column_to_handle] = dataframe[column_to_handle].replace(dict)
  return dataframe

In [ ]:
def handle_null(dataframe, column):
    if (dataframe[column].isnull().any()):
        dataframe[column] = dataframe[column].fillna(dataframe[column].median())
    return dataframe

In [ ]:
def process_gender_age_band(df):
    conditions = [
        (df["GENDER"] == "M") & df["AGE_BAND"].isin(["45-54", "25-34", "35-44"]),
        (df["GENDER"] == "M") & df["AGE_BAND"].isin(["65+", "55-64"]),
        (df["GENDER"] == "M") & (df["AGE_BAND"] == "18-24"),
        (df["GENDER"] == "F") & df["AGE_BAND"].isin(["45-54", "25-34", "35-44"]),
        (df["GENDER"] == "F") & df["AGE_BAND"].isin(["65+", "55-64"]),
        (df["GENDER"] == "F") & (df["AGE_BAND"] == "18-24"),
        (~df["GENDER"].isin(["M", "F"])) & df["AGE_BAND"].isin(["45-54", "25-34", "35-44"]),
        (~df["GENDER"].isin(["M", "F"])) & df["AGE_BAND"].isin(["65+", "55-64"]),
        (~df["GENDER"].isin(["M", "F"])) & (df["AGE_BAND"] == "18-24")
    ]
    choices = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    df["GENDER_AGE_BAND"] = np.select(conditions, choices, default=0)
    return df

In [ ]:
def encode_gross_margin_status(gross_margin):
    if gross_margin < 0:
        return 1
    elif gross_margin == 0:
        return 2
    else:
        return 3

In [ ]:
def handle_season(month):
    if ((month==1) | (month==2) | (month==3)):
        return 1
    elif ((month==4) | (month==5) | (month==6)):
        return 2
    elif ((month==7) | (month==8) | (month==9)):
        return 3
    elif ((month==10) | (month==11) | (month==12)):
        return 4
    else:
        return 0

In [ ]:
def encode_month(df):
  df.loc[df["IS_YEAR"] == 2022, "IS_MONTH"] += 12
  return df

In [ ]:
def is_play_bet(place_bet_value):
    if pd.isna(place_bet_value):
        return 0
    else:
        return 1

In [ ]:
def is_world_cup(date):
    if (date >= "2022-11-20" and date <= "2022-12-18"):
        return 1
    else:
        return 0

#### **C. DEFINE VARIABLES**

In [ ]:
numerical_features = ['AGE', 'TENURE_IN_DAYS', 'FOB_RACING_TURNOVER', 'FOB_SPORT_TURNOVER', 'PARI_RACING_TURNOVER',
                     'PARI_SPORT_TURNOVER', 'TOTAL_TURNOVER', 'DIVIDENDS_PAID', 'TICKETS']

In [ ]:
categorical_features = ["AGE_BAND"]

In [ ]:
numerical_pipeline = Pipeline(steps = [
    ('numerical-scaler', MinMaxScaler())
])

In [ ]:
categorical_ordinal_pipeline = Pipeline(steps = [
    ('ordinal-encoder', OrdinalEncoder())
])

In [ ]:
columns_transform_categorical = ColumnTransformer(transformers = [
    ('categorical_ordinal', categorical_ordinal_pipeline, categorical_features)
],
    remainder = 'drop',
    n_jobs = -1
)

In [ ]:
columns_transform_numerical = ColumnTransformer(transformers = [
    ('numerical', numerical_pipeline, numerical_features)
],
    remainder = 'drop',
    n_jobs = -1
)

In [ ]:
day_of_week_dict = {'Mon' : 2, 'Tue': 3, 'Wed': 4,
                    'Thu': 5, 'Fri': 6, 'Sat': 7, 'Sun': 1}

In [ ]:
segment_dict = {
    'Champions' : 5, 'Loyal Customers' : 4, 'About to sleep' : 3,
    'Hibernating' : 2, 'Lost' : 1
}

In [ ]:
holidays_2021 = [ "2021-01-01", "2021-01-26", "2021-03-01", "2021-04-02", "2021-04-05", "2021-04-25",
                 "2021-04-26", "2021-06-07", "2021-09-27", "2021-12-25", "2021-12-26", "2021-12-27", "2021-12-28"]

In [ ]:
holidays_2022 = [ "2022-01-01", "2022-01-03", "2022-01-26", "2022-03-07", "2022-04-15", "2022-04-17",
                 "2022-04-18", "2022-04-25", "2022-06-06", "2022-09-22", "2022-09-26", "2022-12-25",
                 "2022-12-26", "2022-12-27"]

In [ ]:
holidays_2021_2022 = holidays_2021 + holidays_2022

#### **C. RFM SEGMENT**

In [ ]:
RFM_2021, overall_score_2021 = create_RFM(data, '2021')

In [ ]:
RFM_2022, overall_score_2022 = create_RFM(data, '2022')

In [ ]:
merge_2021 = pd.merge(create_data_by_year_filter(data, 2021), RFM_2021, on='BET_ACCOUNT_NUM_HASH', how = 'inner')
merge_2022 = pd.merge(create_data_by_year_filter(data, 2022), RFM_2022, on='BET_ACCOUNT_NUM_HASH', how = 'inner')

In [ ]:
data = pd.merge(merge_2021, merge_2022, how = 'outer')

In [ ]:
data.to_csv(file_path + 'overview_and_detail.csv')

#### **D. FEATURES ENGINEERING**

##### **C1. CREATE NEW FEATURES**

In [ ]:
data["DATE_DIM"] = pd.to_datetime(data["DATE_DIM"])

In [ ]:
data["IS_YEAR"] = data["DATE_DIM"].dt.year

In [ ]:
data["IS_MONTH"] = data["DATE_DIM"].dt.month

###### **a. SEASONS FEATURES**

In [ ]:
data["SEASON_ENCODE"] = data["IS_MONTH"].apply(handle_season)

In [ ]:
data["IS_SPRING"] = data["SEASON_ENCODE"].apply(lambda x : 1 if x == 1 else 0)

In [ ]:
data["IS_SUMMER"] = data["SEASON_ENCODE"].apply(lambda x : 1 if x == 2 else 0)

In [ ]:
data["IS_AUTUMN"] = data["SEASON_ENCODE"].apply(lambda x : 1 if x == 3 else 0)

In [ ]:
data["IS_WINTER"] = data["SEASON_ENCODE"].apply(lambda x : 1 if x == 4 else 0)

In [ ]:
data = apply_function_in_batches(data, encode_month)

###### **b. GENDER FEATURES**

In [ ]:
data["MALE"] = data["GENDER"].apply(lambda x: 1 if (x == "M") else 0)

In [ ]:
data["FEMALE"] = data["GENDER"].apply(lambda x: 1 if (x == "F") else 0)

In [ ]:
data["UNKNOWN"] = data["GENDER"].apply(lambda x: 1 if (x == "U") else 0)

In [ ]:
data = apply_function_in_batches(data, process_gender_age_band)

###### **c. STATUS FEATURES**

In [ ]:
data["GROSS_MARGIN_STATUS"] = data["GROSS_MARGIN"].apply(encode_gross_margin_status)

###### **d. HOLIDAYS FEATURES**

In [ ]:
data["DATE_DIM"] = data["DATE_DIM"].astype(str)

In [ ]:
data["IS_HOLIDAY"] = data["DATE_DIM"].apply(lambda x: 1 if (x in holidays_2021_2022) else 0)

In [ ]:
data["IS_WORLD_CUP"] = data["DATE_DIM"].apply(is_world_cup)

###### **e. RESIDENTIAL_STATE FEATURES**

In [ ]:
data["IS_WA"] = data["RESIDENTIAL_STATE"].apply(lambda x: 1 if (x == "WA") else 0)

In [ ]:
data["IS_OTH"] = data["RESIDENTIAL_STATE"].apply(lambda x: 1 if (x == "WA") else 0)

###### **f. WEEKDAY FEATURES**

In [ ]:
data["IS_WEEKEND"] = data["DAY_OF_WEEK"].apply(lambda x: 1 if ((x == 7) | (x == 1)) else 0)

In [ ]:
data["IS_WEEKDAY"] = data["IS_WEEKEND"].apply(lambda x: 0 if (x == 1) else 1)

###### **g. YEARS FEATURES**

In [ ]:
data["IS_YEAR_2021"] = data["IS_YEAR"].apply(lambda x: 1 if x == 2021 else 0)

In [ ]:
data["IS_YEAR_2022"] = data["IS_YEAR"].apply(lambda x: 1 if x == 2022 else 0)

###### **h. BET FEATURES**

In [ ]:
data["IS_PLAY_FOB_RACING"] = data["FOB_RACING_TURNOVER"].apply(is_play_bet)

In [ ]:
data["IS_PLAY_FOB_SPORT"] = data["FOB_SPORT_TURNOVER"].apply(is_play_bet)

In [ ]:
data["IS_PLAY_FOB"] = data["IS_PLAY_FOB_RACING"] | data["IS_PLAY_FOB_SPORT"]

In [ ]:
data["IS_PLAY_PARI_RACING"] = data["PARI_RACING_TURNOVER"].apply(is_play_bet)

In [ ]:
data["IS_PLAY_PARI_SPORT"] = data["PARI_SPORT_TURNOVER"].apply(is_play_bet)

In [ ]:
data["IS_PLAY_PARI"] = data["IS_PLAY_PARI_SPORT"] | data["IS_PLAY_PARI_RACING"]

In [ ]:
data["IS_PLAY_SPORT"] = data["IS_PLAY_FOB_SPORT"] | data["IS_PLAY_PARI_SPORT"]

In [ ]:
data["IS_PLAY_RACING"] = data["IS_PLAY_FOB_RACING"] | data["IS_PLAY_PARI_RACING"]

In [ ]:
data["IS_PLAY_BET"] = data["IS_PLAY_FOB"] | data["IS_PLAY_PARI"]

In [ ]:
data["IS_BET_DURING_WORLD_CUP"] = data["IS_PLAY_SPORT"] & data["IS_WORLD_CUP"]

##### **C2. PROCESS EXISTING FEATURES**

In [ ]:
data.drop(columns=["BET_ACCOUNT_NUM_HASH", "GENDER", "RESIDENTIAL_STATE", "DATE_DIM"], inplace=True)

In [ ]:
for column in data.columns:
    data = handle_null(data, column)

In [ ]:
data = transform_columns_in_batches(data, categorical_features, columns_transform_categorical)

In [ ]:
data = transform_columns_in_batches(data, numerical_features, columns_transform_numerical)

In [ ]:
data = handle_column(day_of_week_dict, "DAY_OF_WEEK", data)

In [ ]:
data = handle_column(segment_dict, 'Segment', data)

In [ ]:
data.to_csv(file_path + "ProcessedData.csv", index=False)